In [1]:
from datascience import *
path_data = 'https://github.com/data-8/textbook/raw/gh-pages/data/'
bta = Table.read_table(path_data + 'bta.csv')
bta.show()

Group,Result
Control,1
Control,1
Control,0
Control,0
Control,0
Control,0
Control,0
Control,0
Control,0
Control,0


In [2]:
import numpy as np
bta.group('Group', np.average)

Group,Result average
Control,0.125
Treatment,0.6


In [3]:
observed_outcomes = Table.read_table(path_data + "observed_outcomes.csv")
observed_outcomes.show()

Group,Outcome if assigned treatment,Outcome if assigned control
Control,Unknown,1
Control,Unknown,1
Control,Unknown,0
Control,Unknown,0
Control,Unknown,0
Control,Unknown,0
Control,Unknown,0
Control,Unknown,0
Control,Unknown,0
Control,Unknown,0


In [4]:
bta.group('Group', np.average)

Group,Result average
Control,0.125
Treatment,0.6


In [5]:
observed_proportions = bta.group('Group', np.average).column(1)
observed_distance = abs(observed_proportions.item(0) \
                        - observed_proportions.item(1))
observed_distance

0.475

In [6]:
def distance(table, label, group_label):
    reduced = table.select(label, group_label)
    proportions = reduced.group(group_label, np.average).column(1)
    return abs(proportions.item(1) - proportions.item(0))
    
distance(bta, 'Result', 'Group')

0.475

In [7]:
shuffled_labels = bta.sample(with_replacement=False).column(0)
bta_with_shuffled_labels = bta.with_column('Shuffled Label',
                                           shuffled_labels)
bta_with_shuffled_labels.show()

Group,Result,Shuffled Label
Control,1,Control
Control,1,Control
Control,0,Treatment
Control,0,Treatment
Control,0,Control
Control,0,Control
Control,0,Treatment
Control,0,Treatment
Control,0,Treatment
Control,0,Control


In [8]:
distance(bta_with_shuffled_labels, 'Result', 'Shuffled Label')

0.3

In [9]:
distance(bta_with_shuffled_labels, 'Result', 'Group')

0.475

In [0]:
def one_simulated_distance(table, label, group_label):
    shuffled_labels = table.sample(with_replacement = False
                                      ).column(group_label)
    shuffled_table = table.select(label).with_column(
        'Shuffled Label', shuffled_labels)
    return distance(shuffled_table, label, 'Shuffled Label') 

distances = make_array()
repetitions = 20000
for i in np.arange(repetitions):
    new_distance = one_simulated_distance(bta, 'Result', 'Group')
    distances = np.append(distances, new_distance)    

In [0]:
empirical_P = np.count_nonzero(distances >= observed_distance) / repetitions
empirical_P

In [0]:
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [0]:
Table().with_column('Distance', distances).hist(bins = np.arange(0, 0.7, 0.1))
plots.scatter(observed_distance, 0, color='red', s=40)
plots.title('Prediction Under the Null Hypothesis')
print('Observed Distance', observed_distance)
print('Empirical P-value:', round(empirical_P, 4) *100, '%')